<a href="https://colab.research.google.com/github/Gauravhulmukh/Chatbot-Development-for-Regional-language-using-Artificial-Intelligence/blob/master/Optimized_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
from string import punctuation
stemmer = LancasterStemmer()
nltk.download('averaged_perceptron_tagger')

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
# import our chat-bot intents file
from google.colab import drive
drive.mount('/content/drive/')
import json
with open('/content/drive/My Drive/Colab Notebooks/intents.json') as json_data:
    intents = json.load(json_data)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [17]:

nltk.download('punkt')
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#print("words",nltk.pos_tag(words))
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
22 documents
8 classes ['goodbye', 'greeting', 'instructions', 'payment', 'paymentregistration', 'paymentvalidate', 'startpayment', 'thanks']
38 unique stemmed words ["'s", 'anyon', 'ar', 'be', 'bye', 'cre', 'day', 'don', 'ent', 'for', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'instruct', 'invo', 'is', 'it', 'lat', 'nee', 'new', 'pay', 'pleas', 'reg', 'see', 'submit', 'thank', 'that', 'ther', 'to', 'valid', 'what', 'work', 'would', 'you']


In [0]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    #print("doc",doc)
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

#print("train_x",train_x)
#print("train_y",train_y)

In [19]:

# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 2999  | total loss: 1.25694 | time: 0.007s
| Adam | epoch: 1000 | loss: 1.25694 - acc: 0.8381 -- iter: 16/22
Training Step: 3000  | total loss: 1.80962 | time: 0.010s
| Adam | epoch: 1000 | loss: 1.80962 - acc: 0.7710 -- iter: 22/22
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [0]:
# import our chat-bot intents file
#import json
#with open('intents.json') as json_data:
 #   intents = json.load(json_data)

#model.load('./model.tflearn')

In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [0]:
BOT_MESSAGE=["Hi,i am  Gaurav .How can I help you?"]
EXCEPTION_MESSAGE=["Next"]
#if __name__=="__main__":
botsentence= BOT_MESSAGE
while True:

        if(botsentence==None):
          print("Gaurav>> ",random.choice(EXCEPTION_MESSAGE))
        else:
          print("Gaurav>> ",botsentence)

        usersent=input('You>>> ').strip()
        if usersent =='' or usersent.strip(punctuation).lower() == 'quit' or usersent.strip(punctuation).lower() == 'exit':
            print("Gaurav>> Thank You For Using Me")
            break

        botsentence=response(usersent)
        

Gaurav>>  ['Hi,i am  Gaurav .How can I help you?']
You>>> hello
Good to see you again
Gaurav>>  Next
You>>> hello
Hi there,how can I help?
Gaurav>>  Next


In [0]:
print(botsentence)

None
